# 量化回测系统演示

这是一个完整的量化投资策略回测系统，支持多空组合和纯多头组合的策略测试。

## 系统特点
- 📊 支持股票和指数数据回测
- 🔄 定期再平衡机制
- 📈 多空策略支持
- 📊 丰富的可视化报告
- 📋 详细的交易记录和绩效分析

## 1. 环境准备和依赖安装

In [ ]:
# 安装必要的依赖
import subprocess
import sys

def install_package(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# 检查并安装依赖
required_packages = ['pandas', 'numpy', 'matplotlib', 'seaborn', 'plotly']
for package in required_packages:
    try:
        __import__(package)
        print(f"✅ {package} 已安装")
    except ImportError:
        print(f"📦 安装 {package}...")
        install_package(package)
        print(f"✅ {package} 安装完成")

## 2. 导入系统模块

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# 导入自定义模块
from simulator import Simulator
from visualizer_enhanced import run_enhanced_visualization
from data_generator import DataGenerator

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei', 'Arial Unicode MS']
plt.rcParams['axes.unicode_minus'] = False

print("🚀 系统模块导入成功！")

## 3. 数据准备

In [ ]:
# 检查数据文件
data_file = 'sw_industry_returns.csv'
if os.path.exists(data_file):
    print(f"✅ 找到数据文件: {data_file}")
    # 加载并查看数据
    df = pd.read_csv(data_file)
    print(f"📊 数据维度: {df.shape}")
    print("📅 日期范围:", df['date'].min(), "到", df['date'].max())
    print("🔢 行业数量:", len([col for col in df.columns if col != 'date']))
else:
    print("⚠️ 数据文件不存在，将使用模拟数据")
    
    # 生成模拟数据
    generator = DataGenerator()
    df = generator.generate_data()
    df.to_csv(data_file, index=False)
    print("✅ 模拟数据已生成并保存")

# 显示数据预览
df.head()

## 4. 基础回测演示

In [ ]:
# 创建回测模拟器
simulator = Simulator(
    start_date='2019-01-01',
    end_date='2020-01-01',
    rebalance_freq='monthly',
    initial_nav=1000000.0
)

# 运行回测
print("🔄 开始回测...")
results = simulator.run_backtest(df)
print("✅ 回测完成！")

# 查看结果
for strategy_name, result in results.items():
    if strategy_name != 'validation':
        print(f"\n📊 {strategy_name}策略:")
        print(f"   交易日数: {len(result['history'])}")
        print(f"   交易次数: {len(result['trades'])}")
        if len(result['history']) > 0:
            final_nav = result['history']['nav'].iloc[-1]
            total_return = (final_nav / 1000000.0 - 1) * 100
            print(f"   最终净值: ¥{final_nav:,.2f}")
            print(f"   总收益率: {total_return:.2f}%")

## 5. 增强可视化分析

In [ ]:
# 生成增强可视化
print("📈 生成可视化报告...")
report = run_enhanced_visualization(results, use_plotly=False)
print("✅ 可视化完成！")

# 显示详细报告
import json
with open('simulation_results/detailed_report.json', 'r') as f:
    detailed_report = json.load(f)

print("\n📋 策略绩效报告:")
for strategy, metrics in detailed_report['strategies'].items():
    print(f"\n🎯 {strategy}:")
    for key, value in metrics.items():
        print(f"   {key}: {value}")

## 6. 数据可视化展示

In [ ]:
# 显示生成的图表
from IPython.display import Image, display

# 检查并显示主要图表
chart_files = [
    'simulation_results/performance_comparison.png',
    'simulation_results/cumulative_returns.png',
    'simulation_results/monthly_returns.png',
    'simulation_results/detailed_analysis.png'
]

for chart_file in chart_files:
    if os.path.exists(chart_file):
        print(f"📊 显示图表: {chart_file}")
        display(Image(filename=chart_file))
        print()

## 7. 交易记录分析

In [ ]:
# 加载交易记录
longonly_trades = pd.read_csv('simulation_results/LongOnly_trades.csv')
longshort_trades = pd.read_csv('simulation_results/LongShort_trades.csv')

print("📊 LongOnly策略交易记录:")
print(longonly_trades.head())

print("\n📊 LongShort策略交易记录:")
print(longshort_trades.head())

# 交易统计
print("\n📈 交易统计:")
print(f"LongOnly总交易次数: {len(longonly_trades)}")
print(f"LongShort总交易次数: {len(longshort_trades)}")

# 按日期统计交易
if len(longonly_trades) > 0:
    longonly_trades['date'] = pd.to_datetime(longonly_trades['date'])
    monthly_trades = longonly_trades.groupby(longonly_trades['date'].dt.to_period('M')).size()
    print("\n📅 LongOnly月度交易分布:")
    print(monthly_trades)

## 8. 自定义参数回测

In [ ]:
# 自定义回测参数
custom_simulator = Simulator(
    start_date='2019-06-01',
    end_date='2019-12-31',
    rebalance_freq='weekly',  # 每周再平衡
    initial_nav=500000.0,   # 50万初始资金
)

print("🔄 运行自定义参数回测...")
custom_results = custom_simulator.run_backtest(df)

# 生成新的可视化
custom_report = run_enhanced_visualization(custom_results, use_plotly=False)

print("\n✅ 自定义回测完成！")
print("结果已保存到: simulation_results/")

## 9. 结果文件说明

In [ ]:
import os

# 列出所有结果文件
results_dir = 'simulation_results'
if os.path.exists(results_dir):
    files = os.listdir(results_dir)
    print(f"📁 结果目录 {results_dir} 包含以下文件:")
    
    for file in sorted(files):
        file_path = os.path.join(results_dir, file)
        size = os.path.getsize(file_path)
        print(f"   📄 {file} ({size:,} bytes)")
    
    print("\n📋 文件说明:")
    print("   • *_history.csv - 历史净值数据")
    print("   • *_trades.csv - 交易记录")
    print("   • *.png - 可视化图表")
    print("   • *.json - 详细报告")
    print("   • *.txt - 文本报告")
    print("   • *.csv - 汇总数据")

## 10. 总结与建议

### 🎯 系统功能总结

这个量化回测系统提供了完整的策略测试能力：

**核心功能:**
- ✅ 数据加载和验证
- ✅ 多空策略回测
- ✅ 定期再平衡
- ✅ 绩效计算
- ✅ 可视化报告

**策略类型:**
- 📈 LongOnly: 纯多头策略
- 📊 LongShort: 多空对冲策略

**可视化内容:**
- 📊 累计收益曲线
- 📈 月度收益分布
- 🎯 策略对比
- 📋 详细分析报告